In [1]:
import numpy as np
import progressbar 
import random 
import os
import cv2
import h5py

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, model_from_json
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.constraints import maxnorm
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, SeparableConv2D
from keras.layers import concatenate, add, Lambda
from keras.models import Model
from keras.regularizers import l2

import keras
from keras.applications import imagenet_utils
from keras.applications import Xception
from keras.preprocessing.image import img_to_array

import matplotlib.pyplot as plt
%matplotlib inline

from helpers import HDF5DatasetWriter
from helpers import Utils

Using TensorFlow backend.


In [2]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [ ]:
root = 'datasets/UCF-101-frames-resize/'
include = ['ApplyEyeMakeup', 'ApplyLipstick', 'Archery', 'BabyCrawling', 
           'BalanceBeam', 'BandMarching', 'BaseballPitch', 'Basketball']

In [3]:
output_path = "../input/datasets/UCF-101-hdf5/nosquence-xception-features.hdf5"
batch_size = 32
buffer_size = 1000

In [4]:
db_train = h5py.File("../input/datasets/UCF-101-hdf5/rgbmean_train.hdf5")
db_val = h5py.File("../input/datasets/UCF-101-hdf5/rgbmean_val.hdf5")
db_test = h5py.File("../input/datasets/UCF-101-hdf5/rgbmean_test.hdf5")

x_train = db_train["images"][:]
x_val = db_val["images"][:]
x_test = db_test["images"][:]

y_train = db_train["labels"][:]
y_val = db_val["labels"][:]
y_test = db_test["labels"][:]

In [5]:
num_classes = y_train.shape[1]
num_classes

8

In [6]:
output_path = "../output/"
json_file = open(output_path + 'saved/ucf101_model_xception_99.43.json', 'r')
transfer_model_json = json_file.read()
json_file.close()
transfer_model = model_from_json(transfer_model_json)
transfer_model.load_weights(output_path + "saved/ucf101_weight_xception_99.43.hdf5")

In [7]:
transfer_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 60, 80, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 60, 80, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 60, 80, 32)   128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
separable_conv2d_1 (SeparableCo (None, 60, 80, 32)   1312        batch_normalization_1[0][0]      
__________________________________________________________________________________________________
activation

In [8]:
model = Model(inputs=transfer_model.input, 
              outputs=transfer_model.get_layer('batch_normalization_36').output)

In [9]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 60, 80, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 60, 80, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 60, 80, 32)   128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
separable_conv2d_1 (SeparableCo (None, 60, 80, 32)   1312        batch_normalization_1[0][0]      
__________________________________________________________________________________________________
activation

In [10]:
def extract_feature(model, x, y, output_path, batch_size = 32, buffer_size = 1000):
    dataset = HDF5DatasetWriter((x.shape[0], 8*10*256), y.shape, output_path, bufSize = buffer_size)
    
    widgets = ["Extracting Features: ", progressbar.Percentage(), " ", progressbar.Bar(), " ", progressbar.ETA()]
    pbar = progressbar.ProgressBar(maxval=x.shape[0], widgets=widgets).start()

    for i in np.arange(0, x.shape[0], batch_size):
        batchData = x[i:i + batch_size]
        batchLabels = y[i:i + batch_size]
        batchImages = []
        for (j, image) in enumerate(batchData):
            image = img_to_array(image)
            image = np.expand_dims(image, axis=0)
            image = imagenet_utils.preprocess_input(image)
            batchImages.append(image)
        batchImages = np.vstack(batchImages)
        features = model.predict(batchImages, batch_size=batch_size)
        features = features.reshape((features.shape[0], 8*10*256))
        dataset.add(features, batchLabels)
        pbar.update(i)
    dataset.close()
    pbar.finish()

In [11]:
extract_feature(model, x_train, y_train, output_path_train, batch_size = 32, buffer_size = 1000)

Extracting Features: 100% |#####################################| Time: 0:00:12


In [12]:
extract_feature(model, x_val, y_val, output_path_val, batch_size = 32, buffer_size = 1000)

Extracting Features: 100% |#####################################| Time: 0:00:01


In [13]:
extract_feature(model, x_test, y_test, output_path_test, batch_size = 32, buffer_size = 1000)

Extracting Features: 100% |#####################################| Time: 0:00:01


In [14]:
db_train = h5py.File(output_path_train)

x_train = db_train["images"][:]
y_train = db_train["labels"][:]

In [15]:
x_train.shape

(7501, 20480)

In [16]:
y_train.shape

(7501, 8)